Create RabitMQ connection and channel

In [21]:
import pika
hostName = 'localhost'
helloChannelName = 'Download'
helloRoutingKey = 'Download'

connection = pika.BlockingConnection(pika.ConnectionParameters(host=hostName))
channel = connection.channel()

channel.queue_declare(queue=helloChannelName, durable = True)

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=Download'])>"])>

Send messages

In [22]:
import json
import datetime
from json import JSONEncoder

class DownloadRequestMessage:
    Url = ""
    SourceId = ""
    TimeStamp = datetime.datetime.now()
    
    def __init__(self, url: str, sourceId: str, timeStamp: datetime):
        self.Url = url
        self.SourceId = sourceId
        self.TimeStamp = timeStamp

class DateTimeEncoder(JSONEncoder):
        #Override the default method
        def default(self, obj):
            if isinstance(obj, (datetime.date, datetime.datetime)):
                return obj.isoformat()

messageObj = DownloadRequestMessage(
    url = "https://www.geekwire.com/2023/tech-roadmap-washington-state-quantum-industry/",
    sourceId = "cloudamqp",
    timeStamp = datetime.datetime.now())

jsonMessage = json.dumps(messageObj.__dict__, indent=4, cls=DateTimeEncoder)

print(jsonMessage)

{
    "Url": "https://www.geekwire.com/2023/tech-roadmap-washington-state-quantum-industry/",
    "SourceId": "cloudamqp",
    "TimeStamp": "2023-01-26T14:14:19.329368"
}


In [23]:
channel.basic_publish(exchange='', routing_key = helloRoutingKey, body=jsonMessage)
print(jsonMessage)

{
    "Url": "https://www.geekwire.com/2023/tech-roadmap-washington-state-quantum-industry/",
    "SourceId": "cloudamqp",
    "TimeStamp": "2023-01-26T14:14:19.329368"
}


Close connection. Cleanup resources.

In [24]:
connection.close()